# IMPORT LIBRARY

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
sns.set_style(style='darkgrid')

%matplotlib inline

# PRE-MADE FUNCTIONS

## EVALUATORS

In [ ]:
from sklearn.metrics import classification_report

def eval1(predic,ytest):
  print("-----------------------------------------------------------------")
  print(" hear_attack classification report : \n "+ classification_report(predic,ytest))
  print("-----------------------------------------------------------------")

## MODEL FUNCTIONS

In [ ]:
def lrmod(x_train,y_train,x_test):
  lr1=LogisticRegression()
  lr1.fit(x_train,y_train)
  pred=lr1.predict(x_test)
  return (pred,lr1)

In [ ]:
def svmod(x_train,y_train,x_test,p_grid):
  svg=GridSearchCV(SVC(),param_grid=p_grid,verbose=3,n_jobs=-1,refit=True)
  svg.fit(x_train,y_train)
  pred=svg.predict(x_test)
  return (pred,svg)

## DATA SCALERS

In [ ]:
def transformer1(x_train,x_test):
  m1=MinMaxScaler()
  x_train=m1.fit_transform(x_train)
  x_test=m1.transform(x_test)
  return x_train,x_test

In [ ]:
def transformer2(x_train,x_test):
  m1=StandardScaler()
  x_train=m1.fit_transform(x_train)
  x_test=m1.transform(x_test)
  return x_train,x_test

# GETTING DATA

In [ ]:
data=pd.read_csv('../input/heart-attack-analysis-prediction-dataset/heart.csv')

In [ ]:
data

# DATA DESCRIPTION

In [ ]:
data.info()

In [ ]:
data.describe(include='all')

# NULL CHECK

In [ ]:
data.isna().any()

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(data.isna().transpose(),cmap='viridis',xticklabels=False,cbar=False)

# DATA PREPROCESS

In [ ]:
cpdum=pd.get_dummies(data['cp'],drop_first=True)

In [ ]:
data=pd.concat([data,cpdum],axis=1)

In [ ]:
data.drop(['cp'],axis=1,inplace=True)

# X,Y,SPLITTING

In [ ]:
y=data['output'].values
X=data.drop(['output'],axis=1).values

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=42)

# DATA SCALING

In [ ]:
x_train,x_test=transformer1(x_train,x_test)

# MODEL RUN

## LOGISTIC REGRESSION

In [ ]:
pred1,lrob=lrmod(x_train,y_train,x_test)

## SVC (GRID SEARCH)

In [ ]:
C=[0.5,1,10,50,100,500,1000]
Ga=np.linspace(0.001,1.001,num=30)
kernel=['rbf','linear']
p=dict(C=C,gamma=Ga,kernel=kernel)
pred3,svob=svmod(x_train,y_train,x_test,p)

# EVALUATIONS

## LOGISTIC REGRESSION


In [ ]:
eval1(pred1,y_test)

## SVC

In [ ]:
eval1(pred3,y_test)

svob.best_params_